In [1]:
def outer():
    x = 'python'
    def inner():
        print(x)
    return inner

In [2]:
fn=outer()

In [3]:
fn.__code__.co_freevars

('x',)

In [4]:
fn.__closure__

(<cell at 0x0000025CDE1497C8: str object at 0x0000025CDC421970>,)

In [5]:
def outer():
    x = [1,2,3]
    print(hex(id(x)))
    def inner():
        y = x
        print(hex(id(y)))
    return inner

In [6]:
fn = outer()

0x25cde14e908


In [7]:
fn.__closure__

(<cell at 0x0000025CDE149528: list object at 0x0000025CDE14E908>,)

In [8]:
fn()

0x25cde14e908


In [9]:
def outer():
    c = 0
    def inc():
        nonlocal c
        c += 1
        return c
    return inc

In [10]:
fn = outer()

In [14]:
fn.__closure__, fn.__code__.co_freevars

((<cell at 0x0000025CDE149048: int object at 0x00007FFED84AA190>,), ('c',))

In [12]:
fn()

1

In [13]:
hex(id(1))

'0x7ffed84aa190'

In [15]:
def outer():
    c = 0
    def inc1():
        nonlocal c
        c += 1
        return c
    def inc2():
        nonlocal c
        c += 1
        return c
    return inc1, inc2

In [16]:
fn1, fn2 = outer()

In [17]:
fn1.__code__.co_freevars, fn2.__code__.co_freevars

(('c',), ('c',))

In [18]:
fn1.__closure__, fn2.__closure__

((<cell at 0x0000025CDE149858: int object at 0x00007FFED84AA170>,),
 (<cell at 0x0000025CDE149858: int object at 0x00007FFED84AA170>,))

In [25]:
 def pow(n):
        def inner(x):
            return x ** n
        return inner

In [26]:
square = pow(2)
cube = pow(3)

In [27]:
square(2)

4

In [28]:
def create_adders():
    adders = []
    for n in range(1,4):
        adders.append(lambda x: x+n)
    return adders

In [29]:
adders = create_adders()

In [33]:
adders[0](10),adders[1](10),adders[2](10)

(13, 13, 13)

as n is shared accross (global) so last updated value will be used by all adder function and hence all adder are same value

In [36]:
def create_adders():
    adders = []
    for n in range(1,4):
        adders.append(lambda x, y=n: x+y)
    return adders

In [37]:
adders = create_adders()
adders[0](10),adders[1](10),adders[2](10)

(11, 12, 13)

this will work as we are creating a local variable y and assigning a value, this assignment is done at creation time. hence there is no global variable are defined.

# Closure Applications 

In [38]:
class Averager:
    def __init__(self):
        self.total=0
        self.count=0
        
    def add(number):
        self.total += number
        self.count += 1
        return self.total/self.count

In [39]:
def averager():
    total = 0
    count = 0
    def add(number):
        nonlocal total
        nonlocal count
        total += number
        count += 1
        return total/count
    return add

In [40]:
add_number = averager()
add_number(10)
add_number(20)

15.0

In [41]:
add_number.__closure__


(<cell at 0x0000025CDE149EB8: int object at 0x00007FFED84AA1B0>,
 <cell at 0x0000025CDE149D08: int object at 0x00007FFED84AA530>)

In [43]:
from time import perf_counter

In [44]:
class Timer:
    def __init__(self):
        self.start = perf_counter()
    def poll(self):
        return perf_counter()-self.start

In [45]:
t1 = Timer()

In [46]:
t1.poll()

4.547889299999952

In [47]:
class Timer:
    def __init__(self):
        self.start = perf_counter()
    def __call__(self):
        return perf_counter()-self.start

In [48]:
t2 = Timer()

In [49]:
t2()

3.406148599999142

### Equivatent closure code

In [50]:
def timer():
    start = perf_counter()
    def poll():
        return perf_counter()-start
    return poll

In [51]:
t3 = timer()

In [52]:
t3()

3.4236210999988543

## Closure Application 2

In [54]:
def counter(initial_val=0):
    def inc(incrementer=1):
        nonlocal initial_val
        initial_val += incrementer
        return initial_val
    return inc

In [55]:
counter1 = counter()

In [56]:
counter1()

1

In [57]:
counter1()

2

In [58]:
def counter(fn):
    cnt = 0
    def inner(*args, **kwargs):
        nonlocal cnt
        cnt += 1
        print(f'{fn.__name__} has been called {cnt} times')
        return fn(*args,**kwargs)
    return inner

In [59]:
def add(a,b):
    return a+b
def mul(a,b):
    return a*b

In [60]:
counter_add = counter(add)
counter_mul = counter(mul)

In [61]:
counter_add(10, 20)

add has been called 1 times


30

In [62]:
counter_mul(10, 20)

mul has been called 1 times


200

In [63]:
counter_add(10, 21)

add has been called 2 times


31

In [64]:
counters = dict()


In [65]:
def counter(fn):
    cnt = 0
    def inner(*args, **kwargs):
        nonlocal cnt
        cnt += 1
        #global counters   <-- Optional
        counters[fn.__name__] = cnt
        return fn(*args,**kwargs)
    return inner

In [66]:
counter_add = counter(add)
counter_mul = counter(mul)

In [67]:
counter_add(10,20)
counter_add(14,15)

29

In [68]:
counters

{'add': 2}

In [69]:
counter_mul(12,4)

48

In [70]:
counters

{'add': 2, 'mul': 1}

In [71]:
def counter(fn, counters):
    cnt = 0
    def inner(*args, **kwargs):
        nonlocal cnt
        cnt += 1
        #global counters   <-- Optional
        counters[fn.__name__] = cnt
        return fn(*args,**kwargs)
    return inner

In [72]:
c = dict()

In [73]:
counter_add = counter(add, c)

In [74]:
counter_mul= counter(mul, c)

In [75]:
counter_add(10,20)
counter_add(14,15)
counter_mul(12,4)

48

In [76]:
c

{'add': 2, 'mul': 1}

In [77]:
def fact(n):
    product = 1
    for i in range(2, n+1):
        product *= i
    return product

In [78]:
fact(3)

6

In [79]:
count_fact = counter(fact,c)

In [80]:
count_fact(6)

720

In [81]:
fact = counter(fact,c)

In [86]:
fact(6)

720

In [87]:
c


{'add': 2, 'mul': 1, 'fact': 3}